<a href="https://colab.research.google.com/github/CHATGPTUA/ai-journey/blob/main/notebooks/rag_bot_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === НЕ ТРЕБУЕТ СКАЧИВАНИЯ МОДЕЛЕЙ (работает сразу) ===
!pip -q install scikit-learn pypdf

import urllib.request, re, textwrap, numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Аккуратная разбивка текста на фрагменты
def chunk_text(text, chunk_size=700, overlap=80):
    text = re.sub(r'\s+', ' ', text).strip()
    chunks, start = [], 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        # стараемся резать по границе предложения
        if end < len(text):
            dot = text.rfind('.', start, end)
            if dot != -1 and dot > start + 100:
                end = dot + 1
        chunks.append(text[start:end].strip())
        start = max(end - overlap, end)
    return [c for c in chunks if c]

# 1) Берём твой one-pager.md из GitHub (ничего не меняй)
RAW_URL = "https://raw.githubusercontent.com/CHATGPTUA/ai-journey/main/one-pager.md"
text = urllib.request.urlopen(RAW_URL).read().decode("utf-8", errors="ignore")

# 2) Режем на кусочки и строим TF-IDF индекс
chunks = chunk_text(text, chunk_size=700, overlap=80)
vectorizer = TfidfVectorizer(ngram_range=(1,2))  # униграммы+биграммы
X = vectorizer.fit_transform(chunks)

def retrieve(query, top_k=4):
    q_vec = vectorizer.transform([query])
    scores = linear_kernel(q_vec, X).ravel()  # косинусная близость
    idx = np.argsort(scores)[::-1][:top_k]
    return [(float(scores[i]), chunks[i]) for i in idx]

def answer_extractive(query, top_k=4):
    hits = retrieve(query, top_k=top_k)
    print("\nВопрос:", query)
    print("\nНаиболее релевантные фрагменты:\n")
    for i, (sim, txt) in enumerate(hits):
        print(f"[{i+1}] (сходство={sim:.3f})")
        print(textwrap.fill(txt, 100))
        print()
    print("Подсказка: Сформируй ответ по сути, опираясь на фрагменты выше.\n")

print("Готово! Задавай вопросы по содержимому one-pager.md.")
print("Например: Какие мои цели на 6 месяцев?")
print("Чтобы выйти, напиши: выход\n")

while True:
    q = input("Ваш вопрос: ").strip()
    if q.lower() in ("выход", "exit", "quit"):
        print("Выход. Отличная работа!")
        break
    if not q:
        continue
    answer_extractive(q)


Готово! Задавай вопросы по содержимому one-pager.md.
Например: Какие мои цели на 6 месяцев?
Чтобы выйти, напиши: выход

Ваш вопрос: Какие мои цели на 6 месяцев?

Вопрос: Какие мои цели на 6 месяцев?

Наиболее релевантные фрагменты:

[1] (сходство=0.101)
--- ## 🔑 Мои сильные стороны - Предпринимательское мышление и понимание ценности для клиента. -
Навык коммуникации и умение продавать. - Настойчивость и дисциплина (готов идти шаг за шагом). ---
## 🗓 План на ближайшие 3 месяца - Сделать 2–3 рабочих мини-проекта (бот, дашборд, AI-ассистент). -
Получить первый платёж за внедрение AI-решения. - Начать публиковать кейсы и формировать личный
бренд. --- ## 📊 Метрики успеха - Первый платный пилот у клиента → к концу 2–3 месяца. - Экономия
времени/денег у клиента, подтверждённая кейсом. - Минимум 1–2 публикации в месяц о
прогрессе/результатах. --- ## ✅ Текущий статус - День 1: обучил модель «Hello AI» → точность ~97.5%.

[2] (сходство=0.038)
# 📝 One-pager: мой путь в AI-бизнес ## 🎯 Зачем я иду 